# <center> CS559 Homework#3: Decision Tree and Ensemble Methods</center>
## <center> Due: 11/8/2021 Monday at 11:59 PM</center>


In this assignment, you are going to implement four classifiers - **decision tree, random forest, adaboost, and gradient boost**. 
Then check the performance with `sklearn` built-in algorithms.
In this work, splitting into train and test sets is not necessary. 

The provided data has four columns - three features (a, b, and c) and the target (class). Three features are continuous data and the target is a binary, 0 or 1. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./F21_CS559_HW3_data.csv')

In [3]:
df.head(5)

,a,b,c,class
0,9.4202,-4.3507,10.3764,1
1,9.7044,-4.4601,10.6803,1
2,9.8075,-4.0894,10.6259,1
3,9.2771,-4.0349,10.1166,1
4,9.6447,-3.5968,10.2936,1


### Question 1: Decisition Tree Classifier
- A simple DT implementation (10 pts.)
    - to make the problem simple, implement a decision tree with depth of 3 (the root index is 0).
    - calculate the gini index for each attribute and pick the best attribute for each node.
    - calculate the accuracy using accuracy score. 
- Classification using DecistionTreeClassifier (5 pts)
- Evaluation (5 pts)

In [380]:
def gini_index(y): 
    #print(y.shape)
    p_net = np.unique(y,return_counts=True,axis=0)[1] / y.shape[0]
    #print(p_net, y.shape[1])
    return 1 - np.sum([p_i**2 for p_i in p_net]) # min = 0 if node is pure, max = 0.5 if node has equal distribution

def most_freq_class(y):
    unique_class, freq = np.unique(y, return_counts=True)
    sorted_idx = np.argsort(freq)[::-1]
    sorted_class = unique_class[sorted_idx]
    return sorted_class[0]

class Node():  
    def __init__(self, leaf_value = None, feat_idx = None, alpha = None, node_left = None, node_right = None):    
        self.leaf_value = leaf_value
        if self.leaf_value == None:
            self.feat_idx = feat_idx
            self.alpha = alpha
            self.node_left = node_left
            self.node_right = node_right
            self.dead_end = False
        else:
            self.dead_end = True
            
class my_DecisionTreeClassifier():
    def __init__(self, max_depth = 3):
        self.max_depth = max_depth
        self.root = None
              
    def fit(self, x, y):
        self.root = self.grow(x,y,0)
        
    def grow(self,x,y,depth):
        num_feat = x.shape[1]
        num_class = len(np.unique(y))
        if depth >= self.max_depth or num_class <= 1:
            #print(most_freq_class(y))
            return Node(leaf_value = most_freq_class(y))
        score = 1
        for i in range(num_feat):
            x_i = x[:,i]
            for alpha in x_i:
                idx_left =  np.argwhere(x_i <= alpha)
                idx_right = np.argwhere(x_i > alpha)
                gini_left = gini_index(y[idx_left])
                gini_right = gini_index(y[idx_right])
                left_size = 0
                for a in x_i:
                    if a <= alpha:
                        left_size += 1
                right_size = len(y) - left_size
                weighted_gini_idx = (1/len(y))*((left_size*gini_left)+(right_size*gini_right))
                if weighted_gini_idx < score:
                    score = weighted_gini_idx
                    feat_idx = i
                    alpha_best = alpha
        idx_left =  np.argwhere(x[:,feat_idx] <= alpha_best).squeeze()
        idx_right = np.argwhere(x[:,feat_idx] > alpha_best).squeeze()
        #print(idx_left.shape, idx_right.shape)
        node_left = self.grow(x[idx_left,:],y[idx_left],depth+1)
        node_right = self.grow(x[idx_right,:],y[idx_right],depth+1)
        return Node(feat_idx = feat_idx, alpha = alpha_best, node_left = node_left, node_right = node_right)  
                
    def accuracy(self, x, y):
        yp = self.forward(x)
        return accuracy_score(y,yp)
    
    def forward(self,x):
        return np.array([self.flow(i,self.root) for i in x]).squeeze()
    
    def flow(self,x_i,node):
        if node.dead_end:
            return node.leaf_value
        elif x_i[node.feat_idx] <= node.alpha:
            return self.flow(x_i,node.node_left)
        else:
            return self.flow(x_i,node.node_right)
        
#a = np.matrix((1,1,1,2,2)).T
#gini_index(a)

In [381]:
x = np.array(df[['a','b','c']])
y = np.array(df[['class']]).squeeze()
my_tree = my_DecisionTreeClassifier()
my_tree.fit(x,y)

In [382]:
print('my accuracy =',my_tree.accuracy(x,y))
print(classification_report(y, my_tree.forward(x)))
print(confusion_matrix(y, my_tree.forward(x)))

my accuracy = 0.9996
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1249
           2       1.00      1.00      1.00      1251

    accuracy                           1.00      2500
   macro avg       1.00      1.00      1.00      2500
weighted avg       1.00      1.00      1.00      2500

[[1249    0]
 [   1 1250]]


### Question 2: Random Forest Classifier
- A simle RF implementation (10 pts)
    - make a bootstrap baggin function to make 3 samples.
    - for each sample, run a simple DT from question 1.
    - then average the accuracy. 
- Classification using RandomForestClassifier (5 pts)
- Evaluation (5 pts)

In [377]:
from scipy.stats import mode

def my_bootstrap_baggin(x,y):
    N = len(y)
    idx = np.unique(np.random.choice(N,size=N,replace=True))
    return x[idx], y[idx]

class my_RandomForestClassifier():
    def __init__(self):
        self.n_samples = 3
        self.depth = 3
        self.DT_list = []
        
    def fit(self,x,y):
        for _ in range(self.n_samples):
            DT = DecisionTreeClassifier(max_depth=self.depth)
            xb,yb=my_bootstrap_baggin(x,y)
            DT.fit(xb,yb)
            self.DT_list.append(DT)
            
    def forward(self,x):
        y_raw = np.zeros((x.shape[0], self.n_samples))
        for i in range(self.n_samples):
            y_raw[:,i] = self.DT_list[i].predict(x)
        return self.mode(y_raw)
    
    def mode(self,y_raw):
        return mode(y_raw,axis=1)[0].flatten()
    
    def accuracy(self,x,y):
        yp = self.forward(x)
        return accuracy_score(y,yp)

In [378]:
my_RF = my_RandomForestClassifier()
my_RF.fit(x,y)

In [379]:
x = np.array(df[['a','b','c']])
y = np.array(df[['class']]).flatten()
print('my accuracy =',my_RF.accuracy(x,y))
print(classification_report(y, my_RF.forward(x)))
print(confusion_matrix(y, my_RF.forward(x)))

my accuracy = 0.9992
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1249
           2       1.00      1.00      1.00      1251

    accuracy                           1.00      2500
   macro avg       1.00      1.00      1.00      2500
weighted avg       1.00      1.00      1.00      2500

[[1249    0]
 [   2 1249]]


### Question 3: AdaBoost Classifier
- AB implementation (15 pts)
- Classification using AdaBoostClassifier (5 pts)
- Evaluation (5 pts)

In [374]:
class my_AdaBoostClassifier():
    def __init__(self):
        self.w = None
        self.H = []
        
    def fit(self,x,y):
        #sigma wi = 1/n for idx 0
        self.w = np.ones(x.shape[0])/x.shape[0]
        flag = True
        while flag:
            #calc h and e
            w_temp = self.w
            stump = DecisionTreeClassifier(max_depth=1,max_leaf_nodes=2)
            stump.fit(x,y,sample_weight=w_temp)
            yp = stump.predict(x)
            e = np.sum(w_temp * y!=yp) #/x.shape[0]
            self.H.append(stump)
            if e >= 0.5:
                flag = False
            #calc alpha, Ht+1, wi, Vi
            alpha = (1/2)*np.log((1-e)/e)
            self.w = (self.w*np.exp(-alpha*stump.predict(x)*y))/(2*np.sqrt(e*(1-e)))
        
    def forward(self,x):
        yp_raw = np.array([h.predict(x) for h in self.H])
        return self.mode(yp_raw)
    
    def mode(self,y_raw):
        return mode(y_raw,axis=0)[0].flatten()
    
    def accuracy(self,x,y):
        yp = self.forward(x)
        return accuracy_score(y,yp)

In [375]:
x = np.array(df[['a','b','c']])
y = np.array(df[['class']]).flatten()
y_pass = np.array([-1 if yt == 1 else 1 for yt in y]).flatten()
my_AB = my_AdaBoostClassifier()
my_AB.fit(x,y_pass)

In [376]:
print('my accuracy =',my_AB.accuracy(x,y_pass))
print(classification_report(y_pass, my_AB.forward(x)))
print(confusion_matrix(y_pass, my_AB.forward(x)))

my accuracy = 0.9032
              precision    recall  f1-score   support

          -1       0.84      0.99      0.91      1249
           1       0.99      0.82      0.89      1251

    accuracy                           0.90      2500
   macro avg       0.92      0.90      0.90      2500
weighted avg       0.92      0.90      0.90      2500

[[1236   13]
 [ 229 1022]]


### Question 4: Gradient Boost Classifier
- GB implementation (15 pts)
- Classification using GradientBoostingClassifier (5 pts)
- Evaluation (5 pts)

In [277]:
class my_GradientBoostingClassifier():
    def __init__(self, max_depth = 4):
        self.H = None
        self.depth = max_depth
        
    def fit(self,x,y,epochs=10,alpha=0.01):
        yp = np.full(y.shape, np.mean(y)).flatten()
        for _ in range(epochs):
            t = y - yp
            h = DecisionTreeRegressor(max_depth=self.depth)
            h.fit(x,t)
            self.H = h
            dy = h.predict(x)
            yp -= alpha*dy
        
    def forward(self,x):
        return np.array([2 if ypi > 0 else 1 for ypi in self.H.predict(x)]).flatten()
    
    def accuracy(self,x,y):
        yp =  self.forward(x)
        return accuracy_score(y,yp)

In [278]:
x = np.array(df[['a','b','c']])
y = np.array(df[['class']]).flatten()
my_GB = my_GradientBoostingClassifier()
my_GB.fit(x,y)

In [281]:
print('my accuracy =',my_GB.accuracy(x,y))
print(classification_report(y, my_GB.forward(x)))
print(confusion_matrix(y, my_GB.forward(x)))

my accuracy = 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1249
           2       1.00      1.00      1.00      1251

    accuracy                           1.00      2500
   macro avg       1.00      1.00      1.00      2500
weighted avg       1.00      1.00      1.00      2500

[[1249    0]
 [   0 1251]]
